# Tutorial 08: Multiagent Environments

This tutorial covers and implementation and execution of multiagent environments in Flow. It assumes some level of knowledge or experience in writing custom environments and running experiments with RLlib; for more on these topics see `tutorial07_environments.ipynb` and `tutorial03_rllib.ipynb`, respectively. The rest of the tutorial is organized as follows. Section 1 describes the procedure through which custom environments can be augmented to generate multiagent environments. Then, section 2 walks you through an example of running a multiagent environment
in RLlib.

## 1. Creating a Multiagent Environment Class

In [ ]:
In this part we will be setting up steps to create a multiagent environment in which the agent might use 
a shared or non-shared policy. 

# import the base environment class
from ray.rllib.env import MultiAgentEnv as MultiEnv

# define the environment class, and inherit properties from the Multi-agent version of base env
class MultiAgentEnv(MultiEnv, Env):

# import the environment to the 
from flow.multiagent_envs.multiagent_env import MultiAgentEnv



### 1.1 Shared policies

### 1.2 None-shared policies (FIXME)

In [ ]:
in the flowong exrcise we would see the agents using 2 different policies: standard and the adverserial.
We 


## 2. Running Multiagent Environment in RLlib

When running the scenario that uses multiagent environment, 
we specify certain parameters in the flow_params. flow_param is the dictionary that is called by the 
create_env function which defines the action and bservation space for each agent during the training. 



In [ ]:
flow_params = dict(
    # name of the experiment
    exp_tag='multiagent_figure_eight',

    # name of the flow environment the experiment is running on
    env_name='MultiAgentAccelEnv',

    # name of the scenario class the experiment is running on
    scenario='Figure8Scenario',

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        additional_params={
            'target_velocity': 20,
            'max_accel': 3,
            'max_decel': 3,
            'perturb_weight': 0.03,
            'sort_vehicles': False
        },
    ))


### 2.1 Shared policies

In [ ]:
When we run the shared policy we refer to the same policy for a different agent. In the example below, the agents
will use'av' policy.

    def gen_policy():
        return (PPOPolicyGraph, obs_space, act_space, {})

    # Setup PG with an ensemble of `num_policies` different policy graphs
    policy_graphs = {'av': gen_policy()}

    def policy_mapping_fn(_):
        return 'av'

    config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['av']
        }
    })

    return alg_run, env_name, config

### 2.2 None-shared policies (FIXME)

In [ ]:
When we run the non-shared policy we refer to different policies for each agent. The generate policy will use the 
'av' policy which would be similiar. 
    def gen_policy():
        return (PPOPolicyGraph, obs_space, act_space, {})

    # Setup PG with an ensemble of `num_policies` different policy graphs
    policy_graphs = {'av': gen_policy(), 'adversary': gen_policy()}

    def policy_mapping_fn(agent_id):
        return agent_id

    config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })
    return alg_run, env_name, config